In [34]:
from data_collection import labelize,read_img,rescale
import pandas as pd
import os
from skimage import morphology as mor
import glob
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

In [67]:
def labelize(img_dir, form, save_path):
    labels = []
    weight = []
    image_names = []
    path_list=os.listdir(img_dir)

    for cow_img in path_list:
        for ID in form.Cow_ID:
            if str(ID) in cow_img:
                #here adapt the code to fill the labels you defined yourself
                labels.append(float(form.BCS_average[form.Cow_ID == ID]))
                image_names.append(cow_img)
                weight.append(int(form.Weight[form.Cow_ID == ID]))
    Results = pd.DataFrame({'image_names':image_names, 'BCS':labels, 'weight':weight})

    Results.to_csv(save_path,index=False)
    
    return Results

In [10]:
def baseline(form, baseline_dir, replace, disk):
    
    data = pd.DataFrame({'image_names':form.image_names,'BCS':form.BCS,'weight':form.weight,'angularity':[0]*len(form)})
    img_path = os.path.join(baseline_dir,'*.png')
    
    for i, cow_img  in enumerate(glob.glob(img_path)):
        cow = read_img(cow_img)
        #cow = rescale(cow)
        ball_roll = mor.opening(cow,disk)
        diff = cow-ball_roll
        diff_pd = pd.DataFrame(diff)
        
        diff_pd[diff_pd>1000] = replace
         
        angularity = np.sum(diff)/np.sum(cow)
        data.angularity.iloc[i] = angularity

        
    return diff, data

In [26]:
def fit(X,Y):
    #these are different linear models using only angularity as the attribute
    #Z = np.polyfit(X,Y,1)
    #Z = np.polyfit(X,Y,2)
    
    #adding weight variable
    poly = PolynomialFeatures(2)
    X1 = poly.fit_transform(X)
    Z = LinearRegression(normalize=False,fit_intercept=True).fit(X1,Y)
    return Z.coef_,Z.intercept_

In [27]:
def ang_BCS(angular,weight,Z):
    #BCS = Z[0]*angular*angular + Z[1]*angular +Z[2]
    #BCS = Z[0]*angular + Z[1]
    
    
    #transform inputs to 1, x1, x2, x1^2, x1x2, x2^:
    poly = PolynomialFeatures(2)
    X = np.c_[angular,weight]
    X = poly.fit_transform(X)
    BCS = np.dot(X,Z[0])+Z[1]
    BCS = BCS[0]
    #W = weight/100
    

    return BCS

In [29]:
def rolling_ball(test_dir,BCS_data,Weight_data,Z,disk,replace,err_range):
    
    angularities = []
    BCSs = []
    labels = []
    round_BCSs = []
    round_labels = []

    IDs = []
    errors = []
    weights = []

    im_path = os.path.join(test_dir, '*.png')
    
    for i, cow_img in enumerate(glob.glob(im_path)):
        
        cow = read_img(cow_img)
        ball_roll = mor.opening(cow,disk)
        diff =  cow - ball_roll
        diff_pd = pd.DataFrame(diff)
        diff_pd[diff_pd>1000] = replace
        
        angularity = np.sum(diff)/np.sum(cow)
        
        

    
        for ID in BCS_data.Cow_ID:
            if str(ID) in cow_img:
                #add weight as a factor
                weight = float(Weight_data.Weight[Weight_data.Cow_ID == ID])
                W = weight/1000
                A = angularity*1000
                BCS_res = ang_BCS(A, W, Z)
                
                BCS = format(BCS_res,'.3f')
                round_BCS = round(float(BCS)*4)/4
                
                labels.append(float(BCS_data.BCS_average[BCS_data.Cow_ID == ID]))
                weights.append(weight)
                round_labels.append(round(float(BCS_data.BCS_average[BCS_data.Cow_ID == ID])*4)/4)
                IDs.append(ID)
                
    
        angularities.append(angularity)
        BCSs.append(BCS)
        round_BCSs.append(round_BCS)
        
    BCSs = np.array(BCSs,float)
    labels = np.array(labels,float)
    errors = BCSs - labels
    average_error = np.sum(np.abs(errors))/len(errors)
    qualify = np.count_nonzero(np.abs(errors) < err_range)/len(errors)
    Results = pd.DataFrame({'cow_ID':IDs, 'weight':weights,'angularity':angularities,'label':labels,'estimated_BCS':BCSs,'errors':errors,'round_labels':round_labels,'round_BCS':round_BCSs})
    
    return average_error, qualify, Results

In [137]:
#baseline calculation
baseline_directory = 'C:/Users/BOZ/Zoetis_BCS/BCS_estimation/3D_rolling_ball/baseline/'
BCS_data = pd.read_excel('BCS & Weights from 21Jul21.xlsx',sheet_name = 'BCS')
base_data = labelize(baseline_directory,BCS_data,'C:/Users/BOZ/Zoetis_BCS/BCS_estimation/3D_rolling_ball/baseline.csv')
base_data.sort_values('image_names',inplace=True)
base_data

,image_names,BCS,weight
0,1182_014.png,3.250,625
1,1184_0016.png,2.625,707
2,1185_022.png,2.750,690
3,1187_010.png,3.125,750
4,1188_023.png,2.875,601
5,1191_008.png,3.375,620
6,1196_041.png,2.500,674
7,2158_008.png,3.875,510
8,2184_012.png,3.500,585


In [182]:
replace = 0
disk = mor.disk(8)
diff, baseline_data = baseline(base_data,baseline_directory,replace,disk)
baseline_data.to_csv('baseline.csv')
#baseline_data.sort_values('angularity',inplace=True)
baseline_data

C:\Users\BOZ\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\BOZ\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\BOZ\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

,image_names,BCS,weight,angularity
0,1182_014.png,3.250,625,0.003464
1,1184_0016.png,2.625,707,0.004426
2,1185_022.png,2.750,690,0.004223
3,1187_010.png,3.125,750,0.003529
4,1188_023.png,2.875,601,0.003652
5,1191_008.png,3.375,620,0.003079
6,1196_041.png,2.500,674,0.005265
7,2158_008.png,3.875,510,0.002312
8,2184_012.png,3.500,585,0.002412


In [183]:
X = np.c_[(baseline_data.angularity)*1000,(baseline_data.weight)/1000]
print(X)
#Z  = fit(X,baseline_data.BCS)
#X1 = baseline_data.angularity*1000
Z = fit(X,baseline_data.BCS)
print(Z)

[[3.4643114  0.625     ]
 [4.42612952 0.707     ]
 [4.22266919 0.69      ]
 [3.52948947 0.75      ]
 [3.65152074 0.601     ]
 [3.07855912 0.62      ]
 [5.26514141 0.674     ]
 [2.31182948 0.51      ]
 [2.41186944 0.585     ]]
(array([  0.        ,  -0.34761806, -14.28759376,   0.0632595 ,
        -0.80147643,  12.79653316]), 9.199308603755004)


In [184]:
BCS_data = pd.read_excel('BCS & Weights from 21Jul21.xlsx',sheet_name = 'BCS')
Weight_data = pd.read_excel('BCS & Weights from 21Jul21.xlsx',sheet_name = 'Body Weights')

test_dir = 'C:/Users/BOZ/Zoetis_BCS/BCS_estimation/3D_rolling_ball/test_set/'
ave_error, qualify, Results = rolling_ball(test_dir,BCS_data,Weight_data,Z,disk,replace,err_range=0.)

In [186]:
print("average score error:",format(ave_error,'.3f'))
print("percentage of predictions whose errors lower than 0.5:",format(qualify,'.2%'))

average score error: 0.339
percentage of predictions whose errors lower than 0.5: 67.17%
